# Programming Assignment 4 - Simple Linear vs. Ridge Regression


In the historical heart of Boston, Bob seeks to understand the intricacies of the real estate market. With a linear regression model at his side, Bob wonders if he can improve his predictions. Given your expertise in machine learning, he turns to you for guidance. Specifically, he wants to unravel the factors influencing the median value of homes across different Boston neighborhoods.

To assist Bob, you decide to:
*  Implement the closed-form solution for linear regression. 
* Apply a polynomial transformation to increase model flexibility.
* Utilize ridge regression to control model complexity.
* Apply 10-fold cross-validation for more reliable performance estimates.


Bob is curious and wants to see a comparison between linear and ridge regression, both with and without polynomial transformations, on the same dataset. Thus, the challenge begins!

 Variables in order:
* CRIM:     per capita crime rate by town
*  ZN:       proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS:    proportion of non-retail business acres per town
* CHAS:     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
* NOX:      nitric oxides concentration (parts per 10 million)
* RM:       average number of rooms per dwelling
* AGE:      proportion of owner-occupied units built prior to 1940
* DIS:      weighted distances to five Boston employment centres
* RAD:      index of accessibility to radial highways
* TAX:      full-value property-tax rate per \$10,000
* PTRATIO:  pupil-teacher ratio by town
* B:        $1000(Bk - 0.63)^2$ where Bk is the proportion of blacks by town
* LSTAT:    \% lower status of the population
* MEDV:     Median value of owner-occupied homes in \$1000's

Note: The Boston Housing dataset, especially the 'B' variable, touches upon serious ethical and societal concerns related to race and inequality. Reflect upon these issues, and consider strategies such as excluding the 'B' column from analyses.

With this context, let's assist Bob in his real estate endeavors!


## 1 Setup and Data Preparation
Import Libraries



In [1]:
import numpy as np  # Fundamental package for linear algebra and multidimensional arrays
import pandas as pd  # Data analysis and manipulation tool

# Transform features to polynomial features for model flexibility
from sklearn.preprocessing import PolynomialFeatures  

# Split arrays or matrices into random train and test subsets
from sklearn.model_selection import train_test_split  

# Scale features to zero mean and unit variance, commonly used for normalization
from sklearn.preprocessing import StandardScaler

# Provides train/test indices to split data into train/test sets while performing cross-validation
from sklearn.model_selection import KFold  


Load the Dataset


In [2]:
# Define feature names
# Specifying the names of the columns in our dataset makes it easier to understand and reference them.
feature_names = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "RAD", "PTRATIO", "B", "LSTAT", "MEDV"]

# Load the data
# We read data from a CSV (Comma-Separated Values) file into a DataFrame. DataFrame is a 2D labeled data structure in pandas.
filename = 'Boston_housing.csv'
df = pd.read_csv(filename, sep='\s+', header=None)

# Display basic information about the dataset
# It's good practice to inspect the dataset's size and first few rows to ensure it's loaded correctly and understand its structure.
print("Dataset shape:", df.shape)
print(df.head())

# Extract features and target
# Machine learning typically involves using features (independent variables) to predict a target (dependent variable).
# Here, we separate the dataset into features (X) and target (y).
X = np.array(df.iloc[:, :13])  # All columns up to the 13th are features
y = np.array(df.iloc[:, 13]).reshape(-1, 1)  # The 13th column is our target, and we reshape it to a 2D array for compatibility.

# Preview data
# It's also good practice to preview the data after separation to ensure everything looks as expected.
print("\nFirst 5 rows of X:\n", X[:5])
print("First 5 values of y:\n", y[:5])
print("X shape:", X.shape)
print("y shape:", y.shape)


Dataset shape: (506, 14)
        0     1     2   3      4      5     6       7   8      9     10  \
0  0.00632  18.0  2.31   0  0.538  6.575  65.2  4.0900   1  296.0  15.3   
1  0.02731   0.0  7.07   0  0.469  6.421  78.9  4.9671   2  242.0  17.8   
2  0.02729   0.0  7.07   0  0.469  7.185  61.1  4.9671   2  242.0  17.8   
3  0.03237   0.0  2.18   0  0.458  6.998  45.8  6.0622   3  222.0  18.7   
4  0.06905   0.0  2.18   0  0.458  7.147  54.2  6.0622   3  222.0  18.7   

       11    12    13  
0  396.90  4.98  24.0  
1  396.90  9.14  21.6  
2  392.83  4.03  34.7  
3  394.63  2.94  33.4  
4  396.90  5.33  36.2  

First 5 rows of X:
 [[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 7.

Checking for missing values

After getting the data, it's always a good practice to check for missing values in the dataset. Luckily for us, this dataset has no missing values. Here's how you can verify that:


In [3]:
# 2. Check for Missing Values:
print("Missing values in X:", np.isnan(X).sum())
print("Missing values in y:", np.isnan(y).sum())

Missing values in X: 0
Missing values in y: 0


## Implementing 10-Fold Cross-Validation
With the data now loaded into X and y, your next task is to implement the code to select the optimal regularization and polynomial transformation. Utilize 10-fold cross-validation to assess the various configurations.



## 10-Fold Cross-Validation with Feature Scaling and Polynomial Transformation

Cross-validation is a method to assess the performance of a machine learning model on unseen data by dividing the data into a set number of groups, or "folds".

### Why 10-Fold Cross-Validation?

In 10-fold cross-validation, the dataset is randomly divided into ten parts or folds. The idea is to iteratively train the model on 9 of these folds and test it on the tenth. This is done ten times, once for each fold acting as the validation set. By doing so, we're ensuring that each data point gets to be in a validation set exactly once.

### Feature Scaling Within Cross-Validation

Feature scaling ensures that all features contribute equally to the model performance, which is particularly important for algorithms sensitive to feature magnitudes.

When doing cross-validation, it's crucial that we don't introduce data leakage by scaling using statistics from the entire dataset. Instead:
1. Divide the data into training and validation sets.
2. Fit the scaler on the training set.
3. Apply the scaling to both the training and validation sets using this scaler.

### Polynomial Transformation Within Cross-Validation

Polynomial transformations capture more intricate data relationships by adding polynomial features. Here's how you incorporate it into cross-validation:
1. Divide the data into training and validation sets.
2. Fit the polynomial transformer on the training set.
3. Transform both the training and validation sets using this transformer.
4. Fit the scaler on the transformed training set
4. Apply the scaling to both the transformed training and transformed validation sets using this scaler.

---
### Note on Cross-Validation Error Calculation

In most lecture notes and literature on k-fold cross-validation, the procedure for calculating the cross-validation error typically involves computing the mean of the errors obtained from each fold. However, in the context of our analysis, given the relatively small size of the dataset and the possibility of unequal numbers of samples in each fold, this traditional approach might not be mathematically rigorous.

To address this, our approach for calculating the cross-validation error will deviate slightly from the traditional method. Instead of merely averaging the errors from each fold, we will sum up the errors across all folds and then divide by $ N $, the total number of training examples. This ensures that our error estimate is unbiased and takes into account the potential discrepancy in the number of samples across different folds.

Mathematically, the cross-validation error, $ E_{cv} $, for this assignment is computed as:
$$  E_{\text{cv}} = \frac{1}{N} \sum_{i=1}^{k} \sum_{j \in \text{fold } i} (y^{(j)}- \hat{y}^{(j)})^2
 $$
where $ k $ is the number of folds, $ y^{(j)} $ is the true target value of the $j^{th} $ example, and $ \hat{y}^{(j)} $ is the predicted value for the same example.

---


# Your code goes here

Feel free to add any helper functions you may need.

### Part a) 5-fold Cross Validation using Linear Regression

In [4]:
def linear_regression(X, y):
    # use np.linalg.pinv(a)    
    # Compute the weights using the closed-form solution 
    #### TO-DO #####
    w = np.linalg.pinv(X.T @ X) @ X.T @ y
    
    
    ##############
    return w

 Next implement Squared Error. It measures the average squared difference between the estimated values (predictions) and the actual values (true values). Mathematically, it is represented as: $  \sum_{i=1}^{N} (y^{(i)} - \hat{y}^{(i)})^2 $


In [5]:
def squared_error(y_true, y_pred):    
    #### TO-DO ##### 
    # Calculate the squared differences
    error = (y_true - y_pred) ** 2


    ##############    
    return error

In [6]:
def k_fold_linear_regression(X, y, k=10):
    """
    Perform k-fold cross-validation for linear regression.
    """
    kf = KFold(n_splits=k, random_state=10, shuffle=True)
    #### TO-DO #####
    
    e_in = []
    e_cv = []
    
    
    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        X_train_with_ones_column = np.append(np.ones([len(X_train),1]),X_train,1)
        
        w = linear_regression(X_train_with_ones_column, y_train)
        
        y_pred = X_train_with_ones_column @ w
        
        in_sample_error = squared_error(y_train, y_pred)
        total_in_sample_error_per_fold = np.sum(in_sample_error,0)
        e_in.append(total_in_sample_error_per_fold)
        
        X_val_with_ones_column = np.append(np.ones([len(X_val),1]),X_val,1)
        
        y_pred_val = X_val_with_ones_column @ w
        
        cross_validation_error = squared_error(y_val, y_pred_val)
        total_cross_validation_error_per_fold = np.sum(cross_validation_error,0)
        e_cv.append(total_cross_validation_error_per_fold)

        # Fit the model on training data
        
        
        # Calculate in-sample error and cross-validation error

        
    ##############
    overall_in_sample_error = 0
    for e in e_in:
        overall_in_sample_error += e
    
    average_in_sample_error = overall_in_sample_error / (len(X) * (k-1)) 
    
    overall_cross_validation_error = 0
    for e in e_cv:
        overall_cross_validation_error += e
    final_cv = overall_cross_validation_error/len(X)
    
    return average_in_sample_error, final_cv


In [7]:
e_in, e_cv = k_fold_linear_regression(X,y)
print("For part a)")
print("Average in-sample error using 10-cross-fold validation with linear regression = ", e_in[0])
print("Cross-Validation error = ", e_cv[0])


For part a)
Average in-sample error using 10-cross-fold validation with linear regression =  21.799517678658443
Cross-Validation error =  23.75533358955554


## Using Scaling

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

e_in_scaled, e_cv_scaled = k_fold_linear_regression(X_scaled, y)

print(e_in_scaled, e_cv_scaled)

[21.79951768] [23.75533359]


### Part b) Adding Ridge Regression
Enhance the previous code to include Ridge Regression.

In [9]:
def ridge_regression(X, y, alpha):
    # use np.linalg.pinv(a)    
    # Compute the weights using the closed-form solution 
    #### TO-DO #####
    identity_matrix = np.identity(X.shape[1])
    identity_matrix[0,0] = 0
    
    w = np.linalg.pinv(X.T @ X + len(y)*alpha*identity_matrix) @ X.T @ y
    
    ##############
    return w

In [10]:

def k_fold_ridge_regression(X, y, k=10, lambdas=np.logspace(-5,1,num=15)):
    """
    Perform k-fold cross-validation for ridge regression with various lambda values.
    """
    best_alpha = None
    best_error = float('inf')
    
    kf = KFold(n_splits=k, random_state=10, shuffle=True)
    
    for alpha in lambdas:
    #### TO-DO #####
        e_in = []
        e_cv = []


        for train_index, val_index in kf.split(X):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            X_train_with_ones_column = np.append(np.ones([len(X_train),1]),X_train,1)

            w = ridge_regression(X_train_with_ones_column, y_train, alpha)

            y_pred = X_train_with_ones_column @ w

            in_sample_error = squared_error(y_train, y_pred)
            total_in_sample_error_per_fold = np.sum(in_sample_error,0)
            e_in.append(total_in_sample_error_per_fold)

            X_val_with_ones_column = np.append(np.ones([len(X_val),1]),X_val,1)

            y_pred_val = X_val_with_ones_column @ w

            cross_validation_error = squared_error(y_val, y_pred_val)
            total_cross_validation_error_per_fold = np.sum(cross_validation_error,0)
            e_cv.append(total_cross_validation_error_per_fold)
        
        overall_in_sample_error = 0
        for e in e_in:
            overall_in_sample_error += e

        average_in_sample_error = overall_in_sample_error / (len(X) * (k-1)) 

        overall_cross_validation_error = 0
        for e in e_cv:
            overall_cross_validation_error += e
        final_cv = overall_cross_validation_error/len(X)
        
        print("for alpha (lambda) = ", alpha)
        print("Average in-sample error = ", average_in_sample_error)
        print("Cross-validation error = ", final_cv)
        print("----------------------------------------------------------")
        
        if final_cv < best_error:
            best_error = final_cv
            best_alpha = alpha
    
            
    ##############
    return best_alpha, best_error


In [11]:
#Use your code to answer question b)    
#### TO-DO #####
print("Part b) Average in-sample errors and cross-validation errors for different values of alpha (lambda) - ")
best_alpha , best_error = k_fold_ridge_regression(X_scaled, y)

print("The best cross-validation error was obtained when alpha = " + str(best_alpha) + ".")
print("The cross-validation error in this case is - ", best_error)
    
##############

Part b) Average in-sample errors and cross-validation errors for different values of alpha (lambda) - 
for alpha (lambda) =  1e-05
Average in-sample error =  [21.79951771]
Cross-validation error =  [23.75528201]
----------------------------------------------------------
for alpha (lambda) =  2.6826957952797274e-05
Average in-sample error =  [21.79951788]
Cross-validation error =  [23.75519536]
----------------------------------------------------------
for alpha (lambda) =  7.196856730011514e-05
Average in-sample error =  [21.79951912]
Cross-validation error =  [23.7549638]
----------------------------------------------------------
for alpha (lambda) =  0.00019306977288832496
Average in-sample error =  [21.79952801]
Cross-validation error =  [23.754349]
----------------------------------------------------------
for alpha (lambda) =  0.0005179474679231213
Average in-sample error =  [21.79959149]
Cross-validation error =  [23.75274536]
-----------------------------------------------------

### Part c) Adding Polynomial Transformations and Ridge Regression
Extend their code to incorporate polynomial transformations combined with Ridge Regression.

In [23]:
from sklearn.preprocessing import PolynomialFeatures

def k_fold_poly_ridge(X, y, k=10, lambdas=np.logspace(-5,1,num=15), degrees=[1, 2, 3]):
    """
    Perform k-fold cross-validation for ridge regression with various lambda values and polynomial transformations.
    """
    best_lambda = None
    best_degree = None
    best_error = float('inf')
    
    kf = KFold(n_splits=k, random_state=10, shuffle=True)
    
    for degree in degrees:
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(X)
        for alpha in lambdas:
            e_in = []
            e_cv = []


            for train_index, val_index in kf.split(X_poly):
                X_train, X_val = X_poly[train_index], X_poly[val_index]
                y_train, y_val = y[train_index], y[val_index]

                w = ridge_regression(X_train, y_train, alpha)

                y_pred = X_train @ w

                in_sample_error = squared_error(y_train, y_pred)
                total_in_sample_error_per_fold = np.sum(in_sample_error,0)
                e_in.append(total_in_sample_error_per_fold)

                y_pred_val = X_val @ w

                cross_validation_error = squared_error(y_val, y_pred_val)
                total_cross_validation_error_per_fold = np.sum(cross_validation_error,0)
                e_cv.append(total_cross_validation_error_per_fold)

            overall_in_sample_error = 0
            for e in e_in:
                overall_in_sample_error += e

            average_in_sample_error = overall_in_sample_error / (len(y) * (k-1)) 

            overall_cross_validation_error = 0
            for e in e_cv:
                overall_cross_validation_error += e
            final_cv = overall_cross_validation_error/len(y)

            print("for alpha (lambda) = ", alpha, "and degree = ", degree)
            print("Average in-sample error = ", average_in_sample_error)
            print("Cross-validation error = ", final_cv)
            print("----------------------------------------------------------")
            
            
    #### TO-DO #####
            if final_cv < best_error:
                best_error = final_cv
                best_lambda = alpha
                best_degree = degree
    
    
    return best_lambda, best_degree, best_error


In [25]:
#Use your code to answer question b)    
#### TO-DO #####
print("Part c) and d) Average in-sample errors and cross-validation errors for different values of alpha (lambda) and degrees - ")
best_lambda, best_degree, best_error = k_fold_poly_ridge(X_scaled, y)

print("The best cross-validation error was obtained when alpha = ", best_lambda, "and degree = ", best_degree)
print("The cross-validation error in this case is - ", best_error)
    
##############

Part c) and d) Average in-sample errors and cross-validation errors for different values of alpha (lambda) and degrees - 
for alpha (lambda) =  1e-05 and degree =  1
Average in-sample error =  [21.79951771]
Cross-validation error =  [23.75528201]
----------------------------------------------------------
for alpha (lambda) =  2.6826957952797274e-05 and degree =  1
Average in-sample error =  [21.79951788]
Cross-validation error =  [23.75519536]
----------------------------------------------------------
for alpha (lambda) =  7.196856730011514e-05 and degree =  1
Average in-sample error =  [21.79951912]
Cross-validation error =  [23.7549638]
----------------------------------------------------------
for alpha (lambda) =  0.00019306977288832496 and degree =  1
Average in-sample error =  [21.79952801]
Cross-validation error =  [23.754349]
----------------------------------------------------------
for alpha (lambda) =  0.0005179474679231213 and degree =  1
Average in-sample error =  [21.7995

for alpha (lambda) =  1.389495494373136 and degree =  3
Average in-sample error =  [8.40060909]
Cross-validation error =  [14.92261159]
----------------------------------------------------------
for alpha (lambda) =  3.727593720314938 and degree =  3
Average in-sample error =  [12.18049226]
Cross-validation error =  [18.90536649]
----------------------------------------------------------
for alpha (lambda) =  10.0 and degree =  3
Average in-sample error =  [18.31240598]
Cross-validation error =  [24.47361426]
----------------------------------------------------------
The best cross-validation error was obtained when alpha =  0.19306977288832497 and degree =  3
The cross-validation error in this case is -  [11.4902794]


## Questions and Responses
### Calculation for part b)

In [27]:
feature_values = [1,0.1,11,7,0,0.4,6,70,4,6,300,16,360,10]
print(len(feature_values))
best_alpha = 0.19306977288832497
best_degree = 3

14


In [34]:
# fitting the ridge regression model with alpha = 0.01 (Not using the poly_ridge_function as best degree = 1)
def best_k_fold_poly_ridge(X, y, k=10, alpha = 0.19306977288832497, degrees = 3):
    """
    Perform k-fold cross-validation for ridge regression with various lambda values and polynomial transformations.
    """
    best_lambda = None
    best_degree = None
    best_error = float('inf')
    
    kf = KFold(n_splits=k, random_state=10, shuffle=True)
    
    poly = PolynomialFeatures(degree=degrees)
    X_poly = poly.fit_transform(X)
    e_in = []
    e_cv = []


    for train_index, val_index in kf.split(X_poly):
        X_train, X_val = X_poly[train_index], X_poly[val_index]
        y_train, y_val = y[train_index], y[val_index]

        w = ridge_regression(X_train, y_train, alpha)

        y_pred = X_train @ w

        in_sample_error = squared_error(y_train, y_pred)
        total_in_sample_error_per_fold = np.sum(in_sample_error,0)
        e_in.append(total_in_sample_error_per_fold)

        y_pred_val = X_val @ w

        cross_validation_error = squared_error(y_val, y_pred_val)
        total_cross_validation_error_per_fold = np.sum(cross_validation_error,0)
        e_cv.append(total_cross_validation_error_per_fold)

    overall_in_sample_error = 0
    for e in e_in:
        overall_in_sample_error += e

    average_in_sample_error = overall_in_sample_error / (len(y) * (k-1)) 

    overall_cross_validation_error = 0
    for e in e_cv:
        overall_cross_validation_error += e
    final_cv = overall_cross_validation_error/len(y)

    print("for alpha (lambda) = ", alpha, "and degree = ", degrees)
    print("Average in-sample error = ", average_in_sample_error)
    print("Cross-validation error = ", final_cv)
    print("----------------------------------------------------------")
    
    return w

In [35]:
weights_for_best_model = best_k_fold_poly_ridge(X_scaled, y)
print(weights_for_best_model)

for alpha (lambda) =  0.19306977288832497 and degree =  3
Average in-sample error =  [5.04683739]
Cross-validation error =  [11.4902794]
----------------------------------------------------------
[[ 2.13960140e+01]
 [-4.69212400e-02]
 [ 5.23590246e-02]
 [-5.20811270e-02]
 [ 5.64785355e-03]
 [-1.02072567e-01]
 [ 7.32478263e-01]
 [-4.34884173e-01]
 [-1.38508352e-01]
 [ 1.65249451e-01]
 [-1.01231207e-01]
 [-2.95172037e-01]
 [ 8.07574028e-02]
 [-4.85157234e-01]
 [ 3.33692859e-02]
 [ 2.24194676e-03]
 [-9.99041331e-03]
 [ 6.97589259e-02]
 [-2.47886280e-02]
 [-3.01943672e-01]
 [ 1.13611679e-01]
 [ 6.15319745e-02]
 [-9.68226162e-02]
 [ 1.00918239e-02]
 [ 9.04887919e-02]
 [-3.23249848e-02]
 [ 8.52696009e-02]
 [-1.47803307e-02]
 [-3.05948638e-02]
 [-3.04361882e-02]
 [ 5.45532205e-02]
 [-1.68503529e-01]
 [ 4.73122389e-02]
 [ 3.28900991e-02]
 [-9.33804043e-02]
 [ 1.04111161e-01]
 [ 5.54231911e-02]
 [-1.35381906e-02]
 [ 1.92336624e-01]
 [ 2.10315666e-01]
 [ 2.15809702e-02]
 [-3.12746830e-02]
 [-2.5

In [38]:
#Scaling the given feature values
scaled_feature_values = [1]
poly = PolynomialFeatures(degree=3)
X_poly_feature = poly.fit_transform(feature_values[1:].reshape(len(feature_values)))
print(X_poly_feature.shape)
means = X_poly.mean(axis = 0)
print(means.shape)
standard_deviations = X_poly.std(axis = 0)
print(standard_deviations)

for i in range(1,len(X_poly_feature)):
    scaled_feature_value = (X_poly_feature[i] - means[i]) / standard_deviations[i]
    scaled_feature_values.append(scaled_feature_value)    
    

ValueError: Expected 2D array, got 1D array instead:
array=[1.0e-01 1.1e+01 7.0e+00 0.0e+00 4.0e-01 6.0e+00 7.0e+01 4.0e+00 6.0e+00
 3.0e+02 1.6e+01 3.6e+02 1.0e+01].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [37]:
scaled_features_array = np.array(scaled_feature_values).reshape(1,14)

In [ ]:
predicted_value = scaled_features_array @ weights_for_best_model
print(scaled_predicted_value)

In [ ]:
y_mean = y.mean(axis=0)
y_std = y.std(axis=0)

actual_predicted_value = (scaled_predicted_value * y_std) + y_mean
print("The predicted value with the given input feature values = ", actual_predicted_value[0][0])